In [1]:
!pip install fastai gradio transformers

In [3]:
import gradio as gr
import re
from transformers import pipeline

# Load a more sophisticated NLP model for query understanding
nlp_model = pipeline("text2text-generation", model="t5-small")

# Load NER model for parameter extraction
ner_model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Extended Reservoir formulas and descriptions
formulas = {
    "porosity": {
        "formula": "ϕ = Vp / Vb",
        "description": "Porosity is the ratio of pore volume (Vp) to bulk volume (Vb) of the reservoir rock.",
        "function": lambda Vp, Vb: (Vp / Vb) * 100,  # Porosity in percentage
        "params": ["Vp", "Vb"],
    },
    "initial oil-in-place": {
        "formula": "N = 7758 * A * h * ϕ * Soi",
        "description": "Volumetric method to calculate initial oil-in-place.",
        "function": lambda A, h, ϕ, Soi: 7758 * A * h * ϕ * Soi,
        "params": ["A", "h", "ϕ", "Soi"],
    },
    "water saturation": {
        "formula": "Sw = (1 - Soi)",
        "description": "Water saturation is the complement of oil saturation in the reservoir.",
        "function": lambda Soi: 1 - Soi,
        "params": ["Soi"],
    },
    "bulk volume": {
        "formula": "Vb = A * h",
        "description": "Bulk volume is calculated as surface area times reservoir thickness.",
        "function": lambda A, h: A * h,
        "params": ["A", "h"],
    },
    "hydrocarbon pore volume": {
        "formula": "HCPV = A * h * ϕ * (1 - Sw)",
        "description": "Hydrocarbon pore volume is calculated by multiplying area, thickness, porosity, and water saturation.",
        "function": lambda A, h, ϕ, Sw: A * h * ϕ * (1 - Sw),
        "params": ["A", "h", "ϕ", "Sw"],
    },
    "gor": {
        "formula": "GOR = (Qg / Qo)",
        "description": "Gas-oil ratio (GOR) is the ratio of gas production rate (Qg) to oil production rate (Qo).",
        "function": lambda Qg, Qo: Qg / Qo,
        "params": ["Qg", "Qo"],
    },
    "archie's equation": {
        "formula": "Sw = (a / (Φ^m)) * (Rw / Rt)^n",
        "description": "Archie's equation relates water saturation (Sw) to resistivity and porosity (Φ) parameters.",
        "function": lambda a, Φ, m, Rw, Rt, n: (a / (Φ ** m)) * ((Rw / Rt) ** n),
        "params": ["a", "Φ", "m", "Rw", "Rt", "n"],
    },
    "permeability": {
        "formula": "K = (L / A) * (q / ΔP)",
        "description": "Darcy's law to determine permeability (K).",
        "function": lambda L, A, q, ΔP: (L / A) * (q / ΔP),
        "params": ["L", "A", "q", "ΔP"],
    },
    "oil recovery factor": {
        "formula": "RF = (Soi - Sor) / Soi",
        "description": "Recovery factor is the recoverable oil as a fraction of initial oil in place.",
        "function": lambda Soi, Sor: (Soi - Sor) / Soi,
        "params": ["Soi", "Sor"],
    },
    "residual oil saturation": {
        "formula": "Sor = 1 - (Soi + Sw)",
        "description": "Residual oil saturation is the oil left in the reservoir after primary recovery.",
        "function": lambda Soi, Sw: 1 - (Soi + Sw),
        "params": ["Soi", "Sw"],
    }
}

# Function to extract numerical parameters from query
def extract_parameters(query):
    param_matches = re.findall(r'(\w+)\s*=\s*([\d.]+)', query)
    params = {key: float(value) for key, value in param_matches}
    return params

# Function to understand user intent and extract formula context
def nlp_understand_query(query):
    # Generate a possible interpretation of the query using NLP
    response = nlp_model(query)
    return response[0]["generated_text"]

# Main function to calculate reservoir properties based on query and inputs
def calculate_reservoir_properties(query, Vp=None, Vb=None, A=None, h=None, Soi=None, Sw=None, Qg=None, Qo=None, Φ=None, a=None, m=None, Rw=None, Rt=None, n=None, L=None, q=None, ΔP=None, Sor=None):
    # NLP to understand query intent
    nlp_interpretation = nlp_understand_query(query)

    # Extract parameters from the query using Regex
    params = extract_parameters(query)

    # Update params with manually input values if available
    if Vp is not None: params['Vp'] = Vp
    if Vb is not None: params['Vb'] = Vb
    if A is not None: params['A'] = A
    if h is not None: params['h'] = h
    if Soi is not None: params['Soi'] = Soi
    if Sw is not None: params['Sw'] = Sw
    if Qg is not None: params['Qg'] = Qg
    if Qo is not None: params['Qo'] = Qo
    if Φ is not None: params['Φ'] = Φ
    if a is not None: params['a'] = a
    if m is not None: params['m'] = m
    if Rw is not None: params['Rw'] = Rw
    if Rt is not None: params['Rt'] = Rt
    if n is not None: params['n'] = n
    if L is not None: params['L'] = L
    if q is not None: params['q'] = q
    if ΔP is not None: params['ΔP'] = ΔP
    if Sor is not None: params['Sor'] = Sor

    # Attempt to calculate based on available formulas
    results = []
    for formula_name, formula_data in formulas.items():
        # Check if the required parameters for this formula are available
        if all(param in params for param in formula_data["params"]):
            try:
                result = formula_data["function"](**{param: params[param] for param in formula_data["params"]})
                results.append(f"{formula_name.title()}: {result:.2f} ({formula_data['description']})")
            except Exception as e:
                results.append(f"Error in {formula_name}: {e}")

    # If no results, return NLP interpretation
    if not results:
        return f"No matching formula found. NLP Interpretation: {nlp_interpretation}"

    # Return the list of results
    return "\n".join(results)

# Define Gradio inputs for various parameters
inputs = [
    gr.Textbox(lines=2, label="Query (e.g., 'Vp = 26.17, Vb = 150.80')"),
    gr.Number(label="Pore Volume (Vp)"),
    gr.Number(label="Bulk Volume (Vb)"),
    gr.Number(label="Surface Area (A) in acres"),
    gr.Number(label="Reservoir Thickness (h) in ft"),
    gr.Number(label="Oil Saturation (Soi) as a fraction"),
    gr.Number(label="Water Saturation (Sw) as a fraction"),
    gr.Number(label="Gas Production Rate (Qg)"),
    gr.Number(label="Oil Production Rate (Qo)"),
    gr.Number(label="Porosity (Φ)"),
    gr.Number(label="Constant a"),
    gr.Number(label="Constant m"),
    gr.Number(label="Resistivity of water (Rw)"),
    gr.Number(label="Resistivity of formation (Rt)"),
    gr.Number(label="Constant n"),
    gr.Number(label="Length (L)"),
    gr.Number(label="Flow Rate (q)"),
    gr.Number(label="Pressure Drop (ΔP)"),
    gr.Number(label="Residual Oil Saturation (Sor)")
]

# Define the output for the results
outputs = gr.Textbox(label="Answer")

# Define the Gradio interface
gr.Interface(fn=calculate_reservoir_properties, inputs=inputs, outputs=outputs, title="Reservoir Properties Calculator with NLP").launch(share=True)


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this 

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b06f6d5ea4666754af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
!pip install huggingface_hub


In [7]:
from huggingface_hub import notebook_login
notebook_login()


In [8]:
!git clone https://huggingface.co/spaces/Guilianno/Reservoir-properties-calculator


fatal: destination path 'Reservoir-properties-calculator' already exists and is not an empty directory.


In [9]:
%cd Reservoir-properties-calculator


/content/Reservoir-properties-calculator


In [10]:
with open("requirements.txt", "w") as f:
    f.write("gradio\ntransformers\ntorch\n")


In [11]:
with open("app.py", "w") as file:
    file.write('''import gradio as gr
import re
from transformers import pipeline

# Load a more sophisticated NLP model for query understanding
nlp_model = pipeline("text2text-generation", model="t5-small")

# Load NER model for parameter extraction
ner_model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Extended Reservoir formulas and descriptions
formulas = {
    "porosity": {
        "formula": "ϕ = Vp / Vb",
        "description": "Porosity is the ratio of pore volume (Vp) to bulk volume (Vb) of the reservoir rock.",
        "function": lambda Vp, Vb: (Vp / Vb) * 100,  # Porosity in percentage
        "params": ["Vp", "Vb"],
    },
    "initial oil-in-place": {
        "formula": "N = 7758 * A * h * ϕ * Soi",
        "description": "Volumetric method to calculate initial oil-in-place.",
        "function": lambda A, h, ϕ, Soi: 7758 * A * h * ϕ * Soi,
        "params": ["A", "h", "ϕ", "Soi"],
    },
    "water saturation": {
        "formula": "Sw = (1 - Soi)",
        "description": "Water saturation is the complement of oil saturation in the reservoir.",
        "function": lambda Soi: 1 - Soi,
        "params": ["Soi"],
    },
    "bulk volume": {
        "formula": "Vb = A * h",
        "description": "Bulk volume is calculated as surface area times reservoir thickness.",
        "function": lambda A, h: A * h,
        "params": ["A", "h"],
    },
    "hydrocarbon pore volume": {
        "formula": "HCPV = A * h * ϕ * (1 - Sw)",
        "description": "Hydrocarbon pore volume is calculated by multiplying area, thickness, porosity, and water saturation.",
        "function": lambda A, h, ϕ, Sw: A * h * ϕ * (1 - Sw),
        "params": ["A", "h", "ϕ", "Sw"],
    },
    "gor": {
        "formula": "GOR = (Qg / Qo)",
        "description": "Gas-oil ratio (GOR) is the ratio of gas production rate (Qg) to oil production rate (Qo).",
        "function": lambda Qg, Qo: Qg / Qo,
        "params": ["Qg", "Qo"],
    },
    "archie's equation": {
        "formula": "Sw = (a / (Φ^m)) * (Rw / Rt)^n",
        "description": "Archie's equation relates water saturation (Sw) to resistivity and porosity (Φ) parameters.",
        "function": lambda a, Φ, m, Rw, Rt, n: (a / (Φ ** m)) * ((Rw / Rt) ** n),
        "params": ["a", "Φ", "m", "Rw", "Rt", "n"],
    },
    "permeability": {
        "formula": "K = (L / A) * (q / ΔP)",
        "description": "Darcy's law to determine permeability (K).",
        "function": lambda L, A, q, ΔP: (L / A) * (q / ΔP),
        "params": ["L", "A", "q", "ΔP"],
    },
    "oil recovery factor": {
        "formula": "RF = (Soi - Sor) / Soi",
        "description": "Recovery factor is the recoverable oil as a fraction of initial oil in place.",
        "function": lambda Soi, Sor: (Soi - Sor) / Soi,
        "params": ["Soi", "Sor"],
    },
    "residual oil saturation": {
        "formula": "Sor = 1 - (Soi + Sw)",
        "description": "Residual oil saturation is the oil left in the reservoir after primary recovery.",
        "function": lambda Soi, Sw: 1 - (Soi + Sw),
        "params": ["Soi", "Sw"],
    }
}

# Function to extract numerical parameters from query
def extract_parameters(query):
    param_matches = re.findall(r'(\w+)\s*=\s*([\d.]+)', query)
    params = {key: float(value) for key, value in param_matches}
    return params

# Function to understand user intent and extract formula context
def nlp_understand_query(query):
    # Generate a possible interpretation of the query using NLP
    response = nlp_model(query)
    return response[0]["generated_text"]

# Main function to calculate reservoir properties based on query and inputs
def calculate_reservoir_properties(query, Vp=None, Vb=None, A=None, h=None, Soi=None, Sw=None, Qg=None, Qo=None, Φ=None, a=None, m=None, Rw=None, Rt=None, n=None, L=None, q=None, ΔP=None, Sor=None):
    # NLP to understand query intent
    nlp_interpretation = nlp_understand_query(query)

    # Extract parameters from the query using Regex
    params = extract_parameters(query)

    # Update params with manually input values if available
    if Vp is not None: params['Vp'] = Vp
    if Vb is not None: params['Vb'] = Vb
    if A is not None: params['A'] = A
    if h is not None: params['h'] = h
    if Soi is not None: params['Soi'] = Soi
    if Sw is not None: params['Sw'] = Sw
    if Qg is not None: params['Qg'] = Qg
    if Qo is not None: params['Qo'] = Qo
    if Φ is not None: params['Φ'] = Φ
    if a is not None: params['a'] = a
    if m is not None: params['m'] = m
    if Rw is not None: params['Rw'] = Rw
    if Rt is not None: params['Rt'] = Rt
    if n is not None: params['n'] = n
    if L is not None: params['L'] = L
    if q is not None: params['q'] = q
    if ΔP is not None: params['ΔP'] = ΔP
    if Sor is not None: params['Sor'] = Sor

    # Attempt to calculate based on available formulas
    results = []
    for formula_name, formula_data in formulas.items():
        # Check if the required parameters for this formula are available
        if all(param in params for param in formula_data["params"]):
            try:
                result = formula_data["function"](**{param: params[param] for param in formula_data["params"]})
                results.append(f"{formula_name.title()}: {result:.2f} ({formula_data['description']})")
            except Exception as e:
                results.append(f"Error in {formula_name}: {e}")

    # If no results, return NLP interpretation
    if not results:
        return f"No matching formula found. NLP Interpretation: {nlp_interpretation}"


    # Return the list of results
    return "\\n".join(results)

# Define Gradio inputs for various parameters
inputs = [
    gr.Textbox(lines=2, label="Query (e.g., 'Vp = 26.17, Vb = 150.80')"),
    gr.Number(label="Pore Volume (Vp)"),
    gr.Number(label="Bulk Volume (Vb)"),
    gr.Number(label="Surface Area (A) in acres"),
    gr.Number(label="Reservoir Thickness (h) in ft"),
    gr.Number(label="Oil Saturation (Soi) as a fraction"),
    gr.Number(label="Water Saturation (Sw) as a fraction"),
    gr.Number(label="Gas Production Rate (Qg)"),
    gr.Number(label="Oil Production Rate (Qo)"),
    gr.Number(label="Porosity (Φ)"),
    gr.Number(label="Constant a"),
    gr.Number(label="Constant m"),
    gr.Number(label="Resistivity of water (Rw)"),
    gr.Number(label="Resistivity of formation (Rt)"),
    gr.Number(label="Constant n"),
    gr.Number(label="Length (L)"),
    gr.Number(label="Flow Rate (q)"),
    gr.Number(label="Pressure Drop (ΔP)"),
    gr.Number(label="Residual Oil Saturation (Sor)")
]

# Define the output for the results
outputs = gr.Textbox(label="Answer")

# Define the Gradio interface
gr.Interface(fn=calculate_reservoir_properties, inputs=inputs, outputs=outputs, title="Reservoir Properties Calculator with NLP").launch(share=True)
''')


In [12]:
!git config --global user.email "guiliannofossong@gmail.com"
!git config --global user.name "Guilianno"

In [13]:
!git add app.py
!git commit -m "Fixed unterminated string literal"


[main 1feaeff] Fixed unterminated string literal


In [14]:
!git push


Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 415 bytes | 415.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/spaces/Guilianno/Reservoir-properties-calculator
   d7050ca..1feaeff  main -> main
